# Exercise 2.1 - Computing Stress Fields with Tensors

### Task
Compute the mechanical equilibrium for an element of a displacement field defined in terms of bilinear shape functions.

### Learning goals: 
To familiarize yourself with the PyTorch framework with an example from mechanics

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.autograd import grad

shape functions for bilinear quadrilateral element
$$
\begin{align}
{N}_1(\xi,\eta) = \frac{1}{4}(1-\xi)(1-\eta), \\[8pt]
{N}_2(\xi,\eta) = \frac{1}{4}(1+\xi)(1-\eta), \\[8pt]
{N}_3(\xi,\eta) = \frac{1}{4}(1+\xi)(1+\eta), \\[8pt]
{N}_4(\xi,\eta) = \frac{1}{4}(1-\xi)(1+\eta)
\end{align}
$$

In [ ]:
N1 = lambda xi, eta: 0.25 * (1 - xi) * (1 - eta)
N2 = lambda xi, eta: 0.25 * (1 + xi) * (1 - eta)
N3 = lambda xi, eta: 0.25 * (1 + xi) * (1 + eta)
N4 = lambda xi, eta: 0.25 * (1 - xi) * (1 + eta)

nodal displacements


In [ ]:
U = [0.0, 1.0, 2.0, 1.0]
V = [0.0, 0.0, 0.5, 0.8]
u = (
    lambda xi, eta:
    N1(xi, eta) * U[0]
    + N2(xi, eta) * U[1]
    + N3(xi, eta) * U[2]
    + N4(xi, eta) * U[3]
)
v = (
    lambda xi, eta: N1(xi, eta) * V[0]
                    + N2(xi, eta) * V[1]
                    + N3(xi, eta) * V[2]
                    + N4(xi, eta) * V[3]
)

definition of a spatial grid

In [ ]:
nx = 5
ny = 5

x = torch.linspace(-1, 1, nx, requires_grad=True)
y = torch.linspace(-1, 1, ny, requires_grad=True)
x, y = torch.meshgrid(x, y, indexing="ij")

sampled displacement field

In [ ]:
d = torch.cat((u(x, y).unsqueeze(0), v(x, y).unsqueeze(0)), 0)
# the following is achieved by the concatenation
# d[0, :, :] == u(x,y)
# d[1, :, :] == v(x,y)

gradient computation

In [ ]:
dd_dx = torch.zeros((2, 2, nx, ny))
dd_dx[0, 0] = grad(d[0], x, torch.ones_like(x), create_graph=True, retain_graph=True)[0]
# your code goes here: compute the remaining derivatives

strain computation
$$\mathbf{\epsilon} = \frac{1}{2}(\nabla \mathbf{u} + \nabla \mathbf{u}^\intercal)$$

In [ ]:
# your code goes here: compute the strain field

material properties and 4th order material tensor


In [ ]:
E = 210000.0
nu = 0.3

C = torch.zeros((2, 2, 2, 2))  # 4th order material tensor
C[0, 0, 0, 0] = 1.0
C[0, 0, 1, 1] = nu
C[1, 1, 0, 0] = nu
C[1, 1, 1, 1] = 1.0
C[0, 1, 0, 1] = (1.0 - nu) / 2.0
C = E / (1.0 - nu ** 2) * C

stress computation
$$\mathbf{\sigma} = \mathbf{C} : \mathbf{\epsilon} $$

In [ ]:
# your code goes here: compute the stress field

equilibrium computation

In [ ]:
# your code goes here: compute the equilibrium equations

visualization

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(16, 4))
title = [
    r"$u$",
    r"$-(\nabla \sigma)_{1}$",
    r"$\epsilon_{11}$",
    r"$\epsilon_{22}$",
    r"$\epsilon_{12}$",
    r"$v$",
    r"$-(\nabla \sigma)_{2}$",
    r"$\sigma_{11}$",
    r"$\sigma_{22}$",
    r"$\sigma_{12}$",
]
data = [
    d[0],
    f[0],
    eps[0, 0],
    eps[1, 1],
    eps[0, 1],
    d[1],
    f[1],
    sig[0, 0],
    sig[1, 1],
    sig[0, 1],
]
i = 0
for i in range(2):
    for j in range(5):
        cp = ax[i, j].contourf(
            x.detach(), y.detach(), data[i * 5 + j].detach(), levels=12, cmap=plt.cm.jet
        )
        fig.colorbar(cp, ax=ax[i, j], format="%.3f")
        ax[i, j].set_aspect("equal")
        ax[i, j].set_title(title[i * 5 + j])

fig.tight_layout()
plt.show()